# Health status microsimulation

In this simple model, a person has two states of health: healthy or unhealthy, and may be alive or dead. Let $X_i$ denote the lifetime of individual $i$, health be either 1 for healthy or 0 for ill,  and assume that $\mu$, the hazard of death is Gompertz distributed.

\begin{equation}
X_i \sim Gompertz(a,b)
\end{equation}
that is
\begin{equation}
\mu(x|health=1) = z a \exp(bx)\,,
\end{equation}
where $a$ denotes the death rate at age $x=0$, $b$ the rate of aging and $z$ is an unknown frailty parameter which multiplies the death rate proportionally at all ages.

Furthermore, let

\begin{equation}
\lambda(x|health=1) = k \mu(x)
\end{equation}

be the hazard of transitioning from good health to ill-health. Additionally, an unhealthy person has  $m$ times the hazard of dying than a healthy person:

\begin{equation}
\mu(x|health=0) = m z a \exp(bx)\,.
\end{equation}

Probabilities of falling sick and dying between age $x$ and $(x+1)$ are calculated as

\begin{equation}
p_{ill}(x|health=1) = 1-\exp\left(-\int_x^{x+1}k\mu(x)\,dx\right)\\
p_{death}(x|health=1) = 1-\exp\left(-\int_x^{x+1}\mu(x)\,dx\right)\\
p_{death}(x|health=0) = 1-\exp\left(-\int_x^{x+1}m\mu(x)\,dx\right)\,,
\end{equation}

respectively.

In [64]:
import numpy as np

class Person():
    
    def __init__(self,a,b,z,k,m):
        '''A person is born at age 0, is alive and healthy (ill_age is missing) and has inborn parameters a,b,z,k,m.'''
        self.alive = 1
        self.health = 1
        self.age = 0
        self.k = k
        self.m = m
        self.z = z
        self.a = a
        self.b = b
        self.ill_age = None
    
   
    @property
    def pr_death(self):
        '''Calculates probability of death between age x and (x+1)'''
        a,b,z,m,age = self.a,self.b,self.z,self.m,self.age
        if self.health is 1:
            return 1-np.exp(-z*a/b*(np.exp(b*(age+1))-np.exp(b*age)))
        if self.health is 0:
            return 1-np.exp(-m*z*a/b*(np.exp(b*(age+1))-np.exp(b*age)))
    
    @property    
    def pr_sick(self):
        '''Calculates the probability of transitioning to ill-health between age x and (x+1)'''
        a,b,z,k,age = self.a,self.b,self.z,self.k,self.age
        return 1-np.exp(-k*z*a/b*(np.exp(b*(age+1))-np.exp(b*age)))

In [74]:
def simulate(a,b,z,k,m):
    '''Simulate the life of a person
    
    Parameters:
    -----------
    a: death rate at age x = 0 (Gompertz a)
    b: rate of aging (Gompertz b)
    z: frailty multiplier
    k: ill health transition multiplier
    m: ill health death multiplier
    
    Returns:
    --------
    List of length of 3. First element: age at death, second: age at becoming ill (None if died healthy). third: frailty
    '''
    person = Person(a,b,z,k,m)
    while person.alive is 1:
        chance_sick,chance_death = np.random.uniform(0,1,2)
        if person.health is 1 and chance_sick <= person.pr_sick:
            person.health = 0
            person.ill_age = person.age
        if chance_death <= person.pr_death:
            person.alive = 0
        person.age += 1
    return [person.age,person.ill_age,z]

In [95]:
%matplotlib inline
import matplotlib.pyplot as plt

# store results in a dictionary
result = {}
# number of simulated people
N = 1000
# distribution of frailties
zs = np.random.gamma(5,0.2,N)
for id in range(N):
    result[id] = simulate(a=1e-6,b=.14,z=zs[id],k=1.5,m=3)

In [96]:
import pandas as pd
result = pd.DataFrame.from_dict(result,orient="index")
result.columns = ['death','ill','z']
# to do: groupby or split-apply-combine
# something like
# for age in ages
#    if death > age
#       if ill = None
#        mean(z)
#      else
#        if ill > age
#       mean(z)

groups = result.death.groupby(lambda x: x.year)

,death,ill,z
0,79,NaN,1.648888
1,76,NaN,1.886595
2,73,NaN,0.512546
3,78,76.0,1.379969
4,73,67.0,1.089839
5,84,NaN,0.345289
6,83,78.0,1.227449
7,75,65.0,1.037010
8,81,74.0,1.701833
9,71,NaN,0.742225
